In [ ]:
import pandas as pd
import openpyxl
import re
import numpy as np


In [ ]:
file_path = "data/sales_data.xlsx"
excel_data = pd.read_excel(file_path, sheet_name=None)

dirty_df = excel_data['CA-US']
dirty_df

## We'll define a single function that encapsulates our cleaning steps

This is thie signature. The `pd.DataFrame`s aren't strictly necesssary, they just you _what_ the input and outputs will be
```{python}
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    pass 
    
```

In [ ]:


def remove_non_numbers(value: str, pattern: str="[^(\d|\.|)]") -> float:
    """
    Removes non-numeric characters from a given value and returns the cleaned value as a float.

    Args:
        value (str): The input string to be processed.
        pattern (str): A regular expression pattern used to match non-numeric characters. Defaults to "^[^\d|\.)]" which matches any character that is not a digit, period, or comma.

    Returns:
        float: The cleaned value after removing non-numeric characters. If the input string contains no numeric characters, returns NaN (Not a Number).
        If a number can't be converted

    Examples:
        >>> remove_non_numbers("123")
        123.0
        >>> remove_non_numbers("123abc")
        123.0
        >>> remove_non_numbers("1.2,3")
        1.2
    """
    cleaned_value = re.sub(pattern, "", str(value))
    if cleaned_value == "":
        return np.nan
    try:
        return float(cleaned_value)
    except ValueError as e:
        return -11111111.




def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    General function to clean a pandas DataFrame.

    Args:
        df (pd.DataFrame): Input is a single pandas dataframe.

    Returns:
        pd.DataFrame: The cleaned DataFrame after processing with general cleaning steps.

    Examples:
        >>> clean_data(your_df)
            # Apply the cleaning function to your DataFrame

    Notes:
        - This function may not cover all possible cleaning tasks.
        - You should adapt this function according to your specific needs.
    """
    
    # Lowercase the column names
    df.columns = df.columns.str.lower()
    
    # Trim the DataFrame by removing completely empty rows and columns
    df = df.dropna(how='all', axis=1).dropna(how='all', axis=0)

    # Remove any columns that have a grand total or sub total in the name
    column_row_patterns_to_remove = 'total|grand total|subtotal'
    df = df[df.columns.drop(list(df.filter(regex=column_row_patterns_to_remove)))]
    
    # Find out which rows, if any, have a column with the pattern of a total in it
    mask = df.apply(lambda row: row.str.lower().str.contains(column_row_patterns_to_remove).any(), axis=1)
    # Keep those rows that *don't* have the pattern in it. 
    df = df[~mask]

    # Fix the on_time, it appears that there are a number of patterns indicating yes/no
    # We'll accomplish this with a map.
    true_false_map = {
        "y": True,
        "yes": True,
        "n": False,
        "no": False,
        "N": False,
        "Y": True,
        "Yes": True,
        "No": False,
        1: True,
        0: False,
        "1": True,
        "0": False,
        "true": True,
        "false": False
    }
    df['on_time'] = df['on_time'].map(true_false_map)
    
    # Let remove the duplciates from order id 
    df = df.drop_duplicates(subset='order id', keep='first')
    
    # Every column besides order id, date, and on time should be numbers based
    # on our knowledge of this file. There are a few non-number characters but 
    # in the event that there are no non-number characters, this is a good 
    # step to do anyway, it won't hurt to be cautious. 

    non_number_columns = ['order id','date','on_time']
    number_columns = df.columns.difference(non_number_columns)
    
    df[number_columns] = df[number_columns].map(remove_non_numbers)
    
    # There are some additional values we can get out of order id such as the 
    # year and region.  us do that.
    
    df['region'] = df['order id'].str.extract(r'([A-z]{2})')[0]
    df['year'] = df['order id'].str.extract(r'((?!-)\d{4}(?=-))')[0]
    df['date'] = df['date'] + "/" + df['year']
    
    # Let's unpivot this dataframe so it's easier to aggregate down the line.
    df = df.melt(
        id_vars=['order id', 'date', 'on_time','year','region'],
        value_name='sales value',
        var_name='segment_shipping'
    )

    
    # remove any nans from teh sales value field 
    df = df.dropna(how='any', subset=['sales value'])
    
    
    # Now lets split out the segment_shipping field into it's two variables
    df[['segment', 'shipping_class']] = df['segment_shipping'].str.split('_',expand=True)
    df['shipping_class'] = df['shipping_class'].str.replace("class", "").str.strip()
    df = df.drop('segment_shipping', axis=1)
    
    

    df = df.sort_values(by=['order id'])
    df = df[[
        'order id',
        'date',
        'region',
        'on_time',
        'segment',
        'shipping_class',
        'sales value',
    ]]
    df.columns = [x.replace(' ', '_') for x in df.columns]
    df = df.reset_index(drop=True)
    return df


df = clean_data(dirty_df)
df.head(10)

In [ ]:

excel_data = pd.read_excel(file_path, sheet_name=None)

clean_data_list = []
for sheet_name, data in excel_data.items():
    # print(sheet_name, " - ", data.shape, "rows, columns")
    clean_df = clean_data(data)
    # Adding the name of the sheet to the pandas dataframe
    clean_df['sheet_name'] = sheet_name
    clean_data_list.append(clean_df)
    
combined_df = pd.concat(clean_data_list).reset_index(drop=True)

combined_df.sample(20)

In [ ]:
clean_data_list[1].to_csv('fl-mn.csv')

### Let's add this to a local, file-based database.
- This little database will live in the same folder as this notebook and can be connected to via tableau, excel, and other tools

In [ ]:
import sqlite3

conn = sqlite3.connect('sales.db')
cursor = conn.cursor()

cursor.execute('''
DROP TABLE IF EXISTS sales_table;
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS sales_table (
    order_id TEXT,
    date TEXT,
    region TEXT,
    on_time TEXT,
    segment TEXT,
    shipping_class TEXT,
    sales_value REAL
)
''')

In [ ]:
combined_df.to_sql('sales_table', conn, if_exists='replace', index=False)
conn.close()

In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite3 database
conn = sqlite3.connect('sales.db')

# Read data from the sales table into a pandas DataFrame
df = pd.read_sql_query('SELECT * FROM sales_table', conn)

# Close the database connection
conn.close()

# Display the DataFrame
df

In [ ]:
df.to_parquet("sales_table.parquet")

In [ ]:
df.to_csv("sales_table.csv")